In [43]:
import galois
import numpy as np

p = 71 # Pour tester car sinon c'est long et iliisible
# p = 21888242871839275222246405745257275088548364400416034343698204186575808495617
FP = galois.GF(p)

# input arguments
x = FP(2)
y = FP(3)

v1 = x * x
v2 = y * y
v3 = 5 * x * v1
v4 = 4 * v1 * v2
out = 5*x**3 - 4*x**2*y**2 + 13*x*y**2 + x**2 - 10*y

w = FP([1, out, x, y, v1, v2, v3, v4])

print("w =", w)

R = FP([[0, 0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 5, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 4, 0, 0, 0],
         [0, 0, 13, 0, 0, 0, 0, 0]])

L = FP([[0, 0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0, 1, 0, 0]])

O = FP([[0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 0, 0, 1],
         [0, 1, 0, 10, FP(p - 1), 0, FP(p - 1), 1]])

Lw = np.dot(L, w)
Rw = np.dot(R, w)
Ow = np.dot(O, w)

print("Lw =", Lw)
print("Rw =", Rw)

LwRw = np.multiply(Lw, Rw)

print("Lw * Rw =", LwRw)

print("Ow =     ", Ow)

assert np.all(LwRw == Ow)

w = [ 1 33  2  3  4  9 40  2]
Lw = [2 3 4 9 9]
Rw = [ 2  3 10 16 26]
Lw * Rw = [ 4  9 40  2 21]
Ow =      [ 4  9 40  2 21]


In [44]:
mtxs = [L, R, O]
poly_m = []

for m in mtxs:
    poly_list = []
    for i in range(0, m.shape[1]):
        points_x = FP(np.zeros(m.shape[0], dtype=int))
        points_y = FP(np.zeros(m.shape[0], dtype=int))
        for j in range(0, m.shape[0]):
            points_x[j] = FP(j+1)
            points_y[j] = m[j][i]

        poly = galois.lagrange_poly(points_x, points_y)
        coef = poly.coefficients()[::-1]
        if len(coef) < m.shape[0]:
            coef = np.append(coef, np.zeros(m.shape[0] - len(coef), dtype=int))
        poly_list.append(coef)
    
    poly_m.append(FP(poly_list))

Lp = poly_m[0]
Rp = poly_m[1]
Op = poly_m[2]

print(f'''Lp
{Lp}
''')

print(f'''Rp
{Rp}
''')

print(f'''Op
{Op}
''')

Lp
[[ 0  0  0  0  0]
 [ 0  0  0  0  0]
 [ 5 35  0 29  3]
 [61  6  2 14 59]
 [10 16 30 68 18]
 [67 14 39 31 62]
 [ 0  0  0  0  0]
 [ 0  0  0  0  0]]

Rp
[[ 0  0  0  0  0]
 [ 0  0  0  0  0]
 [68 11 24 50 61]
 [61  6  2 14 59]
 [51 17 20 31 23]
 [ 0  0  0  0  0]
 [ 0  0  0  0  0]
 [ 0  0  0  0  0]]

Op
[[ 0  0  0  0  0]
 [ 1 63 34 41  3]
 [ 0  0  0  0  0]
 [10 62 56 55 30]
 [ 4 43 37 59  0]
 [61  6  2 14 59]
 [ 9 24 67 27 15]
 [67 14 39 31 62]]



In [45]:
U = galois.Poly((w @ Lp)[::-1])
V = galois.Poly((w @ Rp)[::-1])
W = galois.Poly((w @ Op)[::-1])

print("U = ", U)
print("V = ", V)
print("W = ", W)

U =  32x^4 + 12x^3 + 51x^2 + 65x + 55
V =  36x^4 + 53x^3 + 63x^2 + 37x + 26
W =  24x^4 + 40x^3 + 25x^2 + 57


In [46]:
tau = FP(20)

print(f"τ = {tau}")

T = galois.Poly([1, p-1], field=FP)
for i in range(2, L.shape[0] + 1):
    T *= galois.Poly([1, p-i], field=FP)

print("\nT = ", T)
for i in range(1, L.shape[0] + 2):
    print(f"T({i}) = ", T(i))
    if i == L.shape[0]:
        print("-"*10)

T_tau = T(tau)
print(f"\nT(τ) = {T_tau}")

τ = 20

T =  x^5 + 56x^4 + 14x^3 + 59x^2 + 61x + 22
T(1) =  0
T(2) =  0
T(3) =  0
T(4) =  0
T(5) =  0
----------
T(6) =  49

T(τ) = 68


In [47]:
H = (U * V - W) // T
rem = (U * V - W) % T

print("H = ", H)
print("rem = ", rem)

assert rem == 0

H =  16x^3 + 25x^2 + 24x + 14
rem =  0


In [48]:
u = U(tau)
v = V(tau)
_w = W(tau) # sorry, w is taken by witness vector
ht = H(tau)*T_tau

assert u * v - _w == ht, f"{u} * {v} - {_w} != {ht}" # this equation should hold

# Partie 4 

### Coté setup

In [49]:
print("Setup phase")
print("-"*10)
print("Toxic waste:")
tau = FP(20)

print(f"τ = {tau}")

T_tau = T(tau)
print(f"\nT(τ) = {T_tau}")

# ...
from py_ecc.optimized_bn128 import multiply, G1, G2, add, pairing, neg, normalize

# G1[τ^0], G1[τ^1], ..., G1[τ^d]
tau_G1 = [multiply(G1, int(tau**i)) for i in range(0, T.degree)]
# G1[τ^0 * T(τ)], G1[τ^1 * T(τ)], ..., G1[τ^d-1 * T(τ)]
target_G1 = [multiply(G1, int(tau**i * T_tau)) for i in range(0, T.degree - 1)]

# G2[τ^0], G2[τ^1], ..., G2[τ^d]
tau_G2 = [multiply(G2, int(tau**i)) for i in range(0, T.degree)]

print("Trusted setup:")
print("-"*10)
print(f"[τ]G1 = {[normalize(point) for point in tau_G1]}")
print(f"[T(τ)]G1 = {[normalize(point) for point in target_G1]}")

print(f"\n[τ]G2 = {[normalize(point) for point in tau_G2]}")

Setup phase
----------
Toxic waste:
τ = 20

T(τ) = 68
Trusted setup:
----------
[τ]G1 = [(1, 2), (18947110137775984544896515092961257947872750783784269176923414004072777296602, 12292085037693291586083644966434670280746730626861846747147579999202931064992), (15630250211619345980264659554171089045460684386745982646195304578474625146473, 13597371564520873079214108010918820572395693243527082948219837354327653001066), (10609540540875827932797320455850052859827897498153948414964160013685734487046, 4813993645475805825314378837641334666424083557311142113454529369033402933209), (16711416843522114883433203178480474873102412521895441414645328675899158417507, 8853086173564136849925865683598036960975837022999254197357454379453354604209)]
[T(τ)]G1 = [(8387425215776935415463627011971628724627661219697159780852247626883884054341, 4926370050544530674325564905924101320801285406104080171983196085097719394544), (19033251874843656108471242320417533909414939332036131356573128480367742634479, 207921354546080

### Coté Prover

In [50]:
def evaluate_poly(poly, trusted_points, verbose=False):
    coeff = poly.coefficients()[::-1]

    assert len(coeff) == len(trusted_points), "Polynomial degree mismatch!"

    if verbose:
        [print(normalize(point)) for point in trusted_points]

    terms = [multiply(point, int(coeff)) for point, coeff in zip(trusted_points, coeff)]
    evaluation = terms[0]
    for i in range(1, len(terms)):
        evaluation = add(evaluation, terms[i])

    if verbose:
        print("-"*10)
        print(normalize(evaluation))
    return evaluation

print("\nProof generation:")
print("-"*10)
# G1[u0 * τ^0] + G1[u1 * τ^1] + ... + G1[ud-1 * τ^d-1]
A_G1 = evaluate_poly(U, tau_G1)
# G2[v0 * τ^0] + G2[v1 * τ^1] + ... + G2[vd-1 * τ^d-1]
B_G2 = evaluate_poly(V, tau_G2)
# G1[w0 * τ^0] + G1[w1 * τ^1] + ... + G1[wd-1 * τ^d-1]
B_G1 = evaluate_poly(V, tau_G1)
# G1[w0 * τ^0] + G1[w1 * τ^1] + ... + G1[wd-1 * τ^d-1]
Cw_G1 = evaluate_poly(W, tau_G1)
# G1[h0 * τ^0 * T(τ)] + G1[h1 * τ^1 * T(τ)] + ... + G1[hd-2 * τ^d-2 * T(τ)]
HT_G1 = evaluate_poly(H, target_G1)

C_G1 = add(Cw_G1, HT_G1)

print(f"[A]G1 = {normalize(A_G1)}")
print(f"[B]G2 = {normalize(B_G2)}")
print(f"[C]G1 = {normalize(C_G1)}")


Proof generation:
----------
[A]G1 = (7017025200078758986405774325560333190346992234649822631849778427774327985956, 5958546404623823338460408764445293216853207166124345976361550204035544473939)
[B]G2 = ((6025717277188619969774194218275019843478408012016003314468349889098740594070, 10636153660501346958458350109359645700707230832728821207243598043977984638327), (17558649791399501547859830165664567074769931867535519047872130615461993564266, 12877549220549518351367099638512551281844727802331459509720946353383306135709))
[C]G1 = (11131247908022881740571036530800568246102207356973368433630315726932674783054, 7206534061952323513685186539091641199172073138897185427341586707581061450103)


### Coté Verifier

In [51]:
print("\nProof verification:")
print("-"*10)
# e(A, B) == e(C, G2[1])
assert pairing(B_G2, A_G1) == pairing(G2, C_G1), "Pairing check failed!"
print("Pairing check passed!")


Proof verification:
----------


AssertionError: Pairing check failed!

# Partie 5

In [ ]:
def evaluate_poly_list(poly_list, x):
    results = []
    for poly in poly_list:
        results.append(poly(x))
    return results

def print_evaluation(name, results):
    print(f'\n{name} polynomial evaluations:')
    for i in range(0, len(results)):
        print(f'{name}_{i} = {results[i]}')

def to_poly(mtx):
    poly_list = []
    for i in range(0, mtx.shape[0]):
        poly_list.append( galois.Poly(mtx[i][::-1]) )
    return poly_list

def print_poly(name, poly_list):
    print(f'\n{name} polynomials:')
    for i in range(0, len(poly_list)):
        print(f'{name}_{i} = {poly_list[i]}')

print("Setup phase")
print("-"*10)
print("Toxic waste:")
alpha = FP(2)
beta = FP(3)
tau = FP(20)

print(f"α = {alpha}")
print(f"β = {beta}")
print(f"τ = {tau}")

beta_L = beta * Lp
alpha_R = alpha * Rp
K = beta_L + alpha_R + Op # preimage of [βA + αB + C]

Kp = to_poly(K)
print_poly("K", Kp)

print("K evaluations:")
K_eval = evaluate_poly_list(Kp, tau)
print([int(k) for k in K_eval])

Setup phase
----------
Toxic waste:
α = 2
β = 3
τ = 20

K polynomials:
K_0 = 0
K_1 = 3x^4 + 41x^3 + 34x^2 + 63x + 1
K_2 = 60x^4 + 45x^3 + 48x^2 + 56x + 9
K_3 = 41x^4 + 54x^3 + 66x^2 + 21x + 31
K_4 = 29x^4 + 41x^3 + 25x^2 + 54x + 65
K_5 = 32x^4 + 36x^3 + 48x^2 + 48x + 49
K_6 = 15x^4 + 27x^3 + 67x^2 + 24x + 9
K_7 = 62x^4 + 31x^3 + 39x^2 + 14x + 67
K evaluations:
[0, 42, 1, 4, 57, 46, 30, 62]


In [ ]:
from py_ecc.optimized_bn128 import multiply, G1, G2, add, pairing, neg, normalize, eq

# G1[α]
alpha_G1 = multiply(G1, int(alpha))
# G1[β]
beta_G1 = multiply(G1, int(beta))
# G1[τ^0], G1[τ^1], ..., G1[τ^d]
tau_G1 = [multiply(G1, int(tau**i)) for i in range(0, T.degree)]
# G1[βU0(τ) + αV0(τ) + W0(τ)], G1[βU1(τ) + αV1(τ) + W1(τ)], ..., G1[βUd(τ) + αVd(τ) + Wd(τ)]
k_G1 = [multiply(G1, int(k)) for k in K_eval]
# G1[τ^0 * T(τ)], G1[τ^1 * T(τ)], ..., G1[τ^d-1 * T(τ)]
target_G1 = [multiply(G1, int(tau**i * T_tau)) for i in range(0, T.degree - 1)]

# G2[α]
beta_G2 = multiply(G2, int(beta))
# G2[τ^0], G2[τ^1], ..., G2[τ^d-1]
tau_G2 = [multiply(G2, int(tau**i)) for i in range(0, T.degree)]

print("Trusted setup:")
print("-"*10)
print(f"[α]G1 = {normalize(alpha_G1)}")
print(f"[β]G1 = {normalize(beta_G1)}")
print(f"[τ]G1 = {[normalize(point) for point in tau_G1]}")
print(f"[k]G1 = {[normalize(point) for point in k_G1]}")
print(f"[τT(τ)]G1 = {[normalize(point) for point in target_G1]}")

print(f"\n[β]G2 = {normalize(beta_G2)}")
print(f"[τ]G2 = {[normalize(point) for point in tau_G2]}")

Trusted setup:
----------
[α]G1 = (1368015179489954701390400359078579693043519447331113978918064868415326638035, 9918110051302171585080402603319702774565515993150576347155970296011118125764)
[β]G1 = (3353031288059533942658390886683067124040920775575537747144343083137631628272, 19321533766552368860946552437480515441416830039777911637913418824951667761761)
[τ]G1 = [(1, 2), (18947110137775984544896515092961257947872750783784269176923414004072777296602, 12292085037693291586083644966434670280746730626861846747147579999202931064992), (15630250211619345980264659554171089045460684386745982646195304578474625146473, 13597371564520873079214108010918820572395693243527082948219837354327653001066), (10609540540875827932797320455850052859827897498153948414964160013685734487046, 4813993645475805825314378837641334666424083557311142113454529369033402933209), (16711416843522114883433203178480474873102412521895441414645328675899158417507, 8853086173564136849925865683598036960975837022999254197357454379453

### Coté prover

In [ ]:
print("\nProof generation:")
print("-"*10)

U = galois.Poly((w @ Lp)[::-1])
V = galois.Poly((w @ Rp)[::-1])

# G1[u0 * τ^0] + G1[u1 * τ^1] + ... + G1[ud-1 * τ^d-1]
A_G1 = evaluate_poly(U, tau_G1)
# G1[A] = G1[A] + G1[α]
A_G1 = add(A_G1, alpha_G1)
# G2[v0 * τ^0] + G2[v1 * τ^1] + ... + G2[vd-1 * τ^d-1]
B_G2 = evaluate_poly(V, tau_G2)
# G2[B] = G2[B] + G2[β]
B_G2 = add(B_G2, beta_G2)
# G1[h0 * τ^0 * T(τ)] + G1[h1 * τ^1 * T(τ)] + ... + G1[hd-2 * τ^d-2 * T(τ)]
HT_G1 = evaluate_poly(H, target_G1)
assert len(w) == len(k_G1), "Polynomial degree mismatch!"
# w0 * G1[k0] + w1 * G1[k1] + ... + wd-1 * G1[kd-1]
K_G1_terms = [multiply(point, int(scaler)) for point, scaler in zip(k_G1, w)]
K_G1 = K_G1_terms[0]
for i in range(1, len(K_G1_terms)):
    K_G1 = add(K_G1, K_G1_terms[i])

C_G1 = add(HT_G1, K_G1)

print(f"[A]G1 = {normalize(A_G1)}")
print(f"[B]G2 = {normalize(B_G2)}")
print(f"[C]G1 = {normalize(C_G1)}")
print("-" * 10)
print("Verifier uses:")
print(f"[α]G1 = {normalize(alpha_G1)}")
print(f"[β]G1 = {normalize(beta_G1)}")


Proof generation:
----------
[A]G1 = (20201057621909565222688015974061663065890801345394248902888101015073879938174, 15260690854872544540033521081834700467833154363773231429131426606228764242577)
[B]G2 = ((13042121125384136861974056431394866124364898781579936575642795533356734091551, 9713768507479770422998816484884285700180435862667271407681349794085617610483), (2732982957804448731660480125369159110015202240788146056928240163160181308933, 13655728535246490435756951434113857426551031172958877265584860801288365267682))
[C]G1 = (9670159118770557166884645129249860042320569361761535450143898218274202339564, 14618672235222928151465322266525869260518771635325390514956999581783680134810)
----------
Verifier uses:
[α]G1 = (1368015179489954701390400359078579693043519447331113978918064868415326638035, 9918110051302171585080402603319702774565515993150576347155970296011118125764)
[β]G1 = (3353031288059533942658390886683067124040920775575537747144343083137631628272, 19321533766552368860946552437480

#### Coté Verifier (pas possible de faire des additions pour pairing ...)

In [ ]:
# A = A + α
# B = B + β
# C = βA + αB + C
# AB == αβ + [βA + αB + C]
assert pairing(B_G2, A_G1) == pairing(beta_G1, alpha_G1) + pairing(G2, C_G1)


TypeError: Expected an int or FQP object, but got object of type <class 'py_ecc.fields.optimized_bn128_FQ'>

### on va separer w en deux avec public et private

In [ ]:
def split_poly(poly):
    coef = [int(c) for c in poly.coefficients()]
    p1 = coef[-2:]
    p2 = coef[:-2] + [0] * 2

    return galois.Poly(p1, field=FP), galois.Poly(p2, field=FP)

u = U(tau)
v = V(tau)
_w = W(tau) # w taken by witness vector
ht = H(tau)*T_tau

U1, U2 = split_poly(U)
V1, V2 = split_poly(V)
W1, W2 = split_poly(W)

w1 = W1(tau)
w2 = W2(tau)

u1 = U1(tau)
u2 = U2(tau)

v1 = V1(tau)
v2 = V2(tau)

c = (beta * u2 + alpha * v2 + w2) + ht 
k = (beta * u1 + alpha * v1 + w1)

assert (u + alpha) * (v + beta) == alpha * beta + k + c # should be equal

In [ ]:
k_pub_G1, k_priv_G1 = k_G1[:2], k_G1[2:]
pub_input, priv_input = w[:2], w[2:]

print(f"[k_pub]G1 = {[normalize(point) for point in k_pub_G1]}")
print(f"[k_priv]G1 = {[normalize(point) for point in k_priv_G1]}")

print(f"pub_input = {pub_input}")
print(f"priv_input = {priv_input}")

[k_pub]G1 = [(0, 0), (4312786488925573964619847916436127219510912864504589785209181363209026354996, 16161347681839669251864665467703281411292235435048747094987907712909939880451)]
[k_priv]G1 = [(1, 2), (3010198690406615200373504922352659861758983907867017329644089018310584441462, 4027184618003122424972590350825261965929648733675738730716654005365300998076), (5267322610033386327594727284085617807706598503218388887104616381227512437954, 201257782416518842482277204984225354519663728413732211137155795260901992108), (10076202152297490686204017674658616195706856331923113611424582186251671204174, 5332583658353911387952875219880585170702835693592572334525184064653452913461), (1527465159374431915328497116935179161014331322368960485951268517950184093102, 17274044707157828649723710289902216429715848248207037129568326237800068062774), (11685980195759592932437927699188695536499721809655450855081121091338317781569, 21462805816499239816910648235899924534970960585255554399527411795040621885686)]
pub_i

In [ ]:
### Prover

K_priv_G1_terms = [multiply(point, int(scaler)) for point, scaler in zip(k_priv_G1, priv_input)]
K_priv_G1 = K_priv_G1_terms[0]
for i in range(1, len(K_priv_G1_terms)):
    K_priv_G1 = add(K_priv_G1, K_priv_G1_terms[i])

C_G1 = add(HT_G1, K_priv_G1)


### Verifier sur solidity https://mumbai.polygonscan.com/address/0xe529a6ba0847a2e7e2335fbb29ea2eaa3ee00b85#code

# Partie 5

In [ ]:
print("Setup phase")
print("-"*10)
print("Toxic waste:")
alpha = FP(2)
beta = FP(3)
gamma = FP(4)
delta = FP(5)
tau = FP(20)

print(f"α = {alpha}")
print(f"β = {beta}")
print(f"γ = {gamma}")
print(f"δ = {delta}")
print(f"τ = {tau}")

def split_poly(poly):
    coef = [int(c) for c in poly.coefficients()]
    p1 = coef[-2:]
    p2 = coef[:-2] + [0] * 2

    return galois.Poly(p1, field=FP), galois.Poly(p2, field=FP)

u = U(tau)
v = V(tau)
ht = H(tau)*T_tau

U1, U2 = split_poly(U)
V1, V2 = split_poly(V)
W1, W2 = split_poly(W)

w1 = W1(tau)
w2 = W2(tau)

u1 = U1(tau)
u2 = U2(tau)

v1 = V1(tau)
v2 = V2(tau)

c = (beta * u2 + alpha * v2 + w2) * delta**-1 + ht * delta**-1
k = (beta * u1 + alpha * v1 + w1) * gamma**-1

a = u + alpha
b = v + beta

assert a * b == alpha * beta + k * gamma + c * delta # should be equal.

Setup phase
----------
Toxic waste:
α = 2
β = 3
γ = 4
δ = 5
τ = 20


In [ ]:
print("Setup phase")
print("-"*10)
print("Toxic waste:")
alpha = FP(2)
beta = FP(3)
gamma = FP(4)
delta = FP(5)
tau = FP(20)

print(f"α = {alpha}")
print(f"β = {beta}")
print(f"γ = {gamma}")
print(f"δ = {delta}")
print(f"τ = {tau}")

tau_G1 = [multiply(G1, int(tau**i)) for i in range(0, T.degree)]
tau_G2 = [multiply(G2, int(tau**i)) for i in range(0, T.degree)]

powers_tauTtau_div_delta = [(tau**i * T_tau) / delta for i in range(0, T.degree - 1)]
target_G1 = [multiply(G1, int(pTd)) for pTd in powers_tauTtau_div_delta]

assert len(target_G1) == len(H.coefficients()), f"target_G1 length mismatch! {len(target_G1)} != {len(H.coefficients())}"

K_gamma, K_delta = [k/gamma for k in K_eval[:2]], [k/delta for k in K_eval[2:]]
K_gamma_G1 = [multiply(G1, int(k)) for k in K_gamma]
K_delta_G1 = [multiply(G1, int(k)) for k in K_delta]

print("Trusted setup:")
print("-"*10)
print(f"[α]G1 = {normalize(alpha_G1)}")
print(f"[β]G2 = {normalize(beta_G2)}")
print(f"[γ]G2 = {normalize(gamma_G2)}")
print(f"[δ]G2 = {normalize(delta_G2)}")
print(f"[τ]G1 = {[normalize(point) for point in tau_G1]}")
print(f"[τ]G2 = {[normalize(point) for point in tau_G2]}")
print(f"[τT(τ)/δ]G1 = {[normalize(point) for point in target_G1]}")
print(f"[K/γ]G1 = {[normalize(point) for point in K_gamma_G1]}")
print(f"[K/δ]G1 = {[normalize(point) for point in K_delta_G1]}")

Setup phase
----------
Toxic waste:
α = 2
β = 3
γ = 4
δ = 5
τ = 20
Trusted setup:
----------
[α]G1 = (1368015179489954701390400359078579693043519447331113978918064868415326638035, 9918110051302171585080402603319702774565515993150576347155970296011118125764)
[β]G2 = ((2725019753478801796453339367788033689375851816420509565303521482350756874229, 7273165102799931111715871471550377909735733521218303035754523677688038059653), (2512659008974376214222774206987427162027254181373325676825515531566330959255, 957874124722006818841961785324909313781880061366718538693995380805373202866))


NameError: name 'gamma_G2' is not defined

In [ ]:
### Prover

A_G1 = evaluate_poly(U, tau_G1)
A_G1 = add(A_G1, alpha_G1)
B_G2 = evaluate_poly(V, tau_G2)
B_G2 = add(B_G2, beta_G2)
HT_G1 = evaluate_poly(H, target_G1)

# [K/δ*w_priv]G1
Kw_delta_G1_terms = [multiply(point, int(scaler)) for point, scaler in zip(K_delta_G1, w_priv)]
Kw_delta_G1 = Kw_delta_G1_terms[0]
for i in range(1, len(Kw_delta_G1_terms)):
    Kw_delta_G1 = add(Kw_delta_G1, Kw_delta_G1_terms[i])

C_G1 = add(Kw_delta_G1, HT_G1)

### Verifier solidity 